In [2]:
GUI()

# Model Simulation

Here you run the simulation. Either choose a scenario or build your own.

In [1]:
%matplotlib inline
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,7)
import seaborn as sns
import sys
from io import StringIO
from IPython.display import display, Markdown
import time
from model import scenario_substation

def GUI(): 
    #-----------Choose scenario tab----------------------------------
    runbtn = widgets.Button(
            description='Run Simulation',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='', # (FontAwesome names without the `fa-` prefix)
            layout = {'font-size':'24px'}
            )

    scenario_selection = widgets.ToggleButtons(
             options=['1', '2', '3', '4'],
             description='',
             disabled=False,
             button_style='', # 'success', 'info', 'warning', 'danger' or ''
             tooltips=['Description of scenario 1', 'Description of scenario 2', 'Description of scenario 3', 'Description of scenario 4'],
             #icons=['check'] * 4
             layout={'width': 'max-content',
                    'margin':'15px',
                    'margin-bottom': '30px'},)

    chooselayout = widgets.Layout(display='flex',
                        flex_flow='column',
                        align_items='center',
                        border='none',
                        width='100%',
                        )

    scenariobox = widgets.VBox(
        [widgets.HTML(
        value="<h2> Choose a scenario </h2>",
        ),
         scenario_selection, 
         runbtn,],
        layout = chooselayout
        )

    #------------------ Build scenario tab ----------------------------------

    region_dropdown = widgets.Dropdown(options=['Stockholm'],
                                       description='Region:',
                                       disabled=False,
                                      )

    housenew = widgets.IntText(value=0,layout={'width': '70px'})
    housenewbox = widgets.Box([widgets.Label('New houses (electrical heating):'),
                            housenew],)

    houseold = widgets.IntText(value=0,layout={'width': '70px'})
    houseoldbox = widgets.Box([widgets.Label('Old houses (electrical heating):'),
                            houseold],)

    housemix = widgets.IntText(value=0,layout={'width': '70px'})
    housemixbox = widgets.Box([widgets.Label('Mixed age houses (district heating):'),
                            housemix],)

    apart = widgets.IntText(value=0,layout={'width': '70px'})
    apartbox = widgets.Box([widgets.Label('New apartments (district heating):'),
                          apart])

    office = widgets.IntText(value=0,layout={'width': '70px'})
    apartbox = widgets.Box([widgets.Label('New apartments (district heating):'),
                          apart])

    resloadbox = widgets.VBox([housenewbox,
                               houseoldbox,
                               housemixbox,
                               apartbox],
                              layout = {'margin':'15px'}
                              )
    offices = widgets.IntText(value = 0,layout={'width': '70px'})
    officebox = widgets.HBox([widgets.Label('Number of offices:'),
                             offices],
                             layout = {'margin':'15px'}
                            )

    PV69 = widgets.IntText(value=0,
                             min=0,
                             description='69 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PV400 = widgets.IntText(value=0,
                             min=0,
                             description='400 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PV868 = widgets.IntText(value=0,
                             min=0,
                             description='868 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PVbox = widgets.HBox([widgets.Label('Number of solar cells:'),
                         widgets.HBox([PV69,
                                       PV400,
                                       PV868,
                         ])],
                        layout = {'margin':'15px'})
    up = widgets.FileUpload(
                description=" Select .csv",
                accept="", 
                multiple=False)
    
    upbox = widgets.Box([widgets.Label('Upload file for custom loads'),
                         up],
                        layout = {'margin':'15px'})

    EVnumber = widgets.IntText(value = 0, 
                               #description = 'Number of EVs in substation:',
                              layout={'width': '70px'})
    EVparking = widgets.IntText(value = 0, 
                               #description = 'Number of Parking spaces in substation:',
                               layout={'width': '70px'})
    EVkwhkm = widgets.FloatText(value = 0.2, 
                               #description = 'Number of Parking spaces in substation:',
                               layout={'width': '70px'})

    EVbox = widgets.Box([widgets.Label('Number of EVs:'),EVnumber,
                         widgets.Label('Number of parking spaces:'),EVparking,
                         widgets.Label('Mean consumption of EVs (kWh/km):'),EVkwhkm],
                       layout = {'margin':'15px'})

    flexcheck = widgets.Checkbox(value=False,
                                 description='Demand Side Flexibility',
                                 disabled=False,
                                 indent=False
                                )

    flexpercent = widgets.IntText(value=30,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )

    flexdays = widgets.IntText(value=17,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )
    flexloadpercent = widgets.IntText(value=10,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )

    flexbox = widgets.VBox([flexcheck,
                         widgets.HBox([widgets.Label('Flex extent (%)'),flexpercent, 
                         widgets.Label('Days to be flexible/year:'),flexdays, 
                         widgets.Label('Percentage of loads that are flexible:'),flexloadpercent,])
                        ], 
                        layout = {'margin':'15px'})

    EEcheck = widgets.Checkbox(value=False,
                               description='Energy efficiency',
                               disabled=False,
                               indent=False,
                               layout={'font-size': '16px'},
                                )

    EEreduction = widgets.IntSlider(min = 0,
                                    max = 100,
                                    value = 30,
                                    disabled = True, 
                                   )

    EEloads = widgets.IntSlider(min = 0,
                                max = 0,
                                disabled = True,
                               )

    EEbox = widgets.VBox([EEcheck, 
                         widgets.VBox([widgets.HBox([widgets.Label('Number of loads being energy efficient'),EEloads]),
                                       widgets.HBox([widgets.Label('Reduction of consumption(%)'),EEreduction]),
                                      ],
                        layout={'width': 'max-content'},
                         ), 
                         ],
                       layout = {'margin':'15px'})

    runbtn2 = widgets.Button(
            description='Run Simulation',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='', # (FontAwesome names without the `fa-` prefix)
            layout = {'font-size':'24px'}
            )


    btnbox = widgets.HBox([runbtn2],
                        layout = {'width': 'max-content',
                                  'align-items':'stretch',
                                  'margin':'15px'})

    custom_layout = widgets.Layout(display='flex',
                                   flex_flow='column',
                                   align_items='stretch',
                                   border='none',
                                   width='99%',
                        )

    create_scenario = widgets.Box([region_dropdown, 
                                    resloadbox, 
                                    officebox,
                                    upbox,
                                    PVbox,
                                    EVbox, 
                                    EEbox, 
                                    flexbox, 
                                    btnbox,
                                   ],
                                  layout = custom_layout)



    #-------------------Events and Output -----------------------------------
    max_count = 500
    f = widgets.IntProgress(min=0, 
                            max=max_count,
                            description = 'Loading:'
                           ) # instantiate the bar

    def progress(obj):
        display(f) # display the bar
        count = 0
        while count <= max_count:
            f.value += 1 # signal to increment the progress bar
            time.sleep(0.1)
            count += 1

    def run_model(event, scenario = None):
        if scenario:
            print('Running scenario ', scenario)
            region = 'Stockholm'
            # Loads
            office_dict = {800:1,
                          }
            resload_dict = {'HouseNew': 2,
                            'HouseOld' : 2, 
                            'HouseDH': 2, 
                            'ApartmentNewDH': 2
                           }
            # PV
            PV_69 = 0
            PV_400 = 1
            PV_868 = 0
            # EV
            num_EV = 2
            num_parkingloc = 2
            mpg_mu = 0.2
            # Energy Efficiency
            is_efficient = True
            efficiency_percent = 0.3 # Reduce values with 30%
            efficient_loads = 20
            # Flex
            is_flex = True
            flex_days = 17
            percent_flex_loads = 0.30
            flex_reduction = 0.50
            flex_only_noDH = True
            # Custom Loads
            custom = pd.DataFrame()
        else:
            region = 'Stockholm'
            # Loads
            office_dict = {800:offices.value}
            resload_dict = {'HouseNew': housenew.value,
                            'HouseOld': houseold.value,
                            'HouseDH': housemix.value, 
                            'ApartmentNewDH': apart.value,
                           }

            # PV
            PV_69 = PV69.value
            PV_400 = PV400.value
            PV_868 = PV868.value

            # EV
            num_EV = EVnumber.value
            num_parkingloc = EVparking.value

            # EV
            num_EV = EVnumber.value
            num_parkingloc = EVparking.value
            mpg_mu = EVkwhkm.value

            # Energy Efficiency
            is_efficient = EEcheck.value
            efficiency_percent = EEreduction.value/100
            efficient_loads = EEloads.value

            # Flex
            is_flex = flexcheck.value
            flex_days = flexdays.value
            percent_flex_loads = flexloadpercent.value/100
            flex_reduction = flexpercent.value/100
            flex_only_noDH = True
            
            # Custom Loads
            custom = df_converter()

        substation = scenario_substation(region,
                               resload_dict, 
                               office_dict, 
                               PV_69, 
                               PV_400, 
                               PV_868, 
                               num_EV,
                               num_parkingloc,
                               mpg_mu,
                               is_efficient, 
                               efficiency_percent, 
                               efficient_loads, 
                               is_flex, 
                               flex_days, 
                               percent_flex_loads, 
                               flex_reduction,
                               flex_only_noDH,
                               custom)
        
        
        nameinfobox = widgets.Box([widgets.HTML(value='<h3> Results of a simulation with 1 substation. </h3>')],
                     layout = {'margin':'15px'})

        consumptioninfobox = widgets.VBox([widgets.HTML(value='Average consumption [kWh/h]: {} (+/- {})'.format(substation.mu, substation.sigma)),
                                       widgets.HTML(value='Max consumption [kWh]: {}'.format(substation.dataframe['AggregatedLoad'].max())), 
                                       widgets.HTML(value='Total consumption [kWh]: {}'.format(substation.dataframe['AggregatedLoad'].sum())),
                                       widgets.HTML(value='Based on data from {} to {}.'.format(substation.start,substation.end)),
                                       widgets.HTML(value='The peak consumption occurs at: {}'.format(substation.find_max())),
                                      ],
                                     layout = {'margin':'15px'})

        countinfobox = widgets.VBox([widgets.HTML(value='Number of houses: {}'.format(substation.house_count)),
                                 widgets.HTML(value='Number of apartments: {}'.format(substation.apartment_count)),
                                 widgets.HTML(value='Number of offices: {}'.format(substation.office_count)),
                                 widgets.HTML(value='Number of loads with DH: {}'.format(substation.DH_count)),
                                 widgets.HTML(value='Number of EVs: {}'.format(substation.EV_count)),
                                 widgets.HTML(value='Number of EV charging locations: {}'.format(num_parkingloc)),],
                               layout = {'margin':'15px'})
        
        pvinfobox = widgets.Box([widgets.HTML(value='Number of PVs: {}'.format(substation.PV_count))])
        
        eeinfobox = widgets.Box([])
        flexinfobox = widgets.Box([])

        trendinfobox = widgets.VBox([pvinfobox, 
                                     eeinfobox, 
                                     flexinfobox],
                               layout = {'margin':'15px'})

        numinfobox = widgets.VBox([nameinfobox, 
                                   consumptioninfobox, 
                                   countinfobox, 
                                   trendinfobox
                                  ])
        
        loadcurve = widgets.Output()
        durationcurve = widgets.Output()
        monthplot = widgets.Output()
        hourplot = widgets.Output()
        weekdayplot = widgets.Output()
        graphinfobox = widgets.VBox(children = [loadcurve,
                                                durationcurve,
                                                monthplot,
                                                hourplot,
                                                weekdayplot])
        with loadcurve:
            fig1 = plt.plot(substation.dataframe['AggregatedLoad'])
            plt.title('Aggregated load curve')
            plt.ylabel('kWh')
            plt.show()
        with durationcurve:
            fig2 = substation.plot_load_duration_curve()
            plt.show()
        with monthplot:
            substation.create_date_cols()
            ax = sns.boxplot(data=substation.dataframe, x='Month', y='AggregatedLoad')
            ax.set_ylabel('kWh')
            ax.set_title('Consumption by month')
            plt.show()
        with hourplot:
            ax = sns.boxplot(data=substation.dataframe, x='Hour', y='AggregatedLoad')
            ax.set_ylabel('kWh')
            ax.set_title('Consumption by hour')
            plt.show()
        with weekdayplot:
            order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
            ax1 = sns.boxplot(data=substation.dataframe, x='Weekday', y='AggregatedLoad', order=order)
            ax1.set_ylabel('kWh')
            ax1.set_xlabel('')
            ax1.set_title('Consumption by day of the week')
            plt.show()
            

        tabout = widgets.Tab([numinfobox,graphinfobox])
        tabout.set_title(0, 'Info')
        tabout.set_title(1, 'Plots and Graphs')
        display(tabout)
        
        


    runbtn.on_click(lambda e: run_model(e,scenario = scenario_selection.value))
    runbtn2.on_click(lambda e: run_model(e))

    def flex_disable(event):
        if flexpercent.disabled:
            flexpercent.disabled = False
            flexdays.disabled = False
            flexloadpercent.disabled = False

        else:
            flexpercent.disabled = True
            flexdays.disabled = True
            flexloadpercent.disabled = True

    flexcheck.observe(lambda e: flex_disable(e))

    def ee_disable(event):
        if EEloads.disabled:
            EEloads.disabled = False
            EEreduction.disabled = False
        else:
            EEloads.disabled = True
            EEreduction.disabled = True 

    EEcheck.observe(lambda e: ee_disable(e))

    def load_change(event):
        change = event['new'] - event['old']
        if EEloads.max + change > 0:
            EEloads.max += change

    housenew.observe(lambda e: load_change(e), names='value')
    houseold.observe(lambda e: load_change(e), names='value')
    housemix.observe(lambda e: load_change(e), names='value')
    apart.observe(lambda e: load_change(e), names='value')
    offices.observe(lambda e: load_change(e), names='value')
    
    def content_parser():
        if up.value == {}:
            return    
        else:
            typ, content = "", ""
            up_value = up.value
            for i in up_value.keys():
                typ = up_value[i]["metadata"]["type"]
                if typ == "text/csv":
                    content = up_value[i]["content"]
                    content_str = str(content, 'utf-8')

               # if eraser.value != {}: 
               #     for val in eraser.value:
               #         if val == "tab":
               #             content_str = content_str.replace("\t","")
               #         else:
               #             content_str = content_str.replace(val,"")
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io
            
    def df_converter():
        content = content_parser()
        if content is not None:
            df = pd.read_csv(content, sep=',', index_col=0, parse_dates = True)    
            return df
        else:
            return None


    #----------------- Tabs and display --------------------------------------

    tab = widgets.Tab([scenariobox,create_scenario])
    tab.set_title(0, 'Scenario 1 - 4')
    tab.set_title(1, 'Custom Scenario')

    display(Markdown('# Model Simulation'), Markdown('Here you run the simulation. Either choose a scenario or build your own.'))
    display(tab)

In [ ]:
file = widgets.FileUpload(
    description='Upload .csv',
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

def upload_file(args):
    upload_filename = list(args['new'].keys())[0]
    uploaded_file = args['new'][upload_filename]
    print(uploaded_file["content"])
    
    
file.observe(upload_file, names=['value'])
#display(file)

In [3]:
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
tab = widgets.Tab()    
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(
                description=" Select .csv",
                accept="", 
                multiple=False)
delim = widgets.RadioButtons(
                options=[',', ';', ' '],
                description='Separator: ',
                disabled=False)
eraser = widgets.SelectMultiple(
                options=['tab','"'],
                value=['tab'],
                #rows=10,
                description='Delimiter: ',
                disabled=False)
rows = widgets.IntSlider(
                value=0,
                step=1,
                description='# of lines:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='d')
button_upload = widgets.Button(
                description='Upload',
                disabled=False,
                button_style='warning',
                tooltip='Click to Upload',
                icon='check')
button_preview = widgets.Button(
                description='Preview',
                disabled=False,
                button_style='info',
                tooltip='Click to Preview',
                icon='search')
button_plot = widgets.Button(
                description='Plot',
                disabled=False,
                button_style='danger',
                tooltip='Click to Plot',
                icon='pencil')
graph_type = widgets.Dropdown(
                options=['Bar Chart', 'Line Chart'],
                value='Bar Chart',
                description='Chart Type:',
                disabled=False)
x_axis = widgets.Dropdown(
                options=[''],
                value='',
                description='X-Axis:',
                disabled=False)
y_axis = widgets.Dropdown(
                options=[''],
                value='',
                description='Y-Axis:',
                disabled=False)
color_picker = widgets.ColorPicker(
                concise=False,
                description='Color Picker: ',
                value='lightblue',
                disabled=False)
toggle = widgets.ToggleButtons(
                options=['Preview  ', 'Info  ', 'Stats  '],
                description='Options',
                disabled=False,
                button_style='warning',
                icons=['search', 'info', 'calculator'])
accordion = widgets.Accordion(children=[
                up, 
                widgets.VBox([delim, eraser]), 
                rows])
accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')
accordion_box = widgets.VBox([
                accordion, 
                widgets.HBox([button_preview, button_upload]),
                out
                ])
children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    widgets.VBox([
        widgets.HBox([graph_type, color_picker]),
        widgets.HBox([x_axis, y_axis]), 
        button_plot,
        out
        ])]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")
display(tab)
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
            
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None
    
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n --------Your uploaded DF looks like:-------- \n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')
    
    
def plot():
    graph = graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                graph))
            if (df is not None):
                df = df.head(50)
                height = df[y_axis.value]
                bars = df[x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':
                    plt.bar(
                        y_pos, 
                        height, 
                        color=color_picker.value)
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height, 
                        color=color_picker.value,
                        marker='o', 
                        linestyle='solid'
                    )
                    plt.xticks(bars)
                plt.show()
                
def preview_clicked(b):
    preview()
def upload_clicked(b):
    upload()
def desc_clicked(b):
    desc()
def plotter_clicked(b):
    plot()
    
button_preview.on_click(preview_clicked)
button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')
button_plot.on_click(plotter_clicked)